In [4]:
from sklearn.externals import joblib

In [1]:
from scipy import sparse
import csv
import sys
import os
import pandas
import numpy as np
import multiprocessing

In [2]:
X = []
Y = []
f1=open(unicode('./datasetPersonTab'))
f2=open(unicode('./datasetNNPTab'))
for i, line in enumerate(f1):
    if (i == 0):
        continue
    X.append(line.strip().split('\t')[4:])
    Y.append('Person')
size = len(X)
np.random.shuffle(X)
for i, line in enumerate(f2):
    if (i == 0):
        continue
    X.append(line.strip().split('\t')[4:])
    Y.append('No')
np.random.shuffle(X[size:])

In [103]:
X_test = []
Y_test = []
f1=open(unicode('./datasetPersonTabTest'))
f2=open(unicode('./datasetNNPTabTest'))
for i, line in enumerate(f1):
    if (i == 0):
        continue
    X_test.append(line.strip().split('\t')[4:])
    Y_test.append('Person')
size_test = len(X_test)
for i, line in enumerate(f2):
    if (i == 0):
        continue
    X_test.append(line.strip().split('\t')[4:])
    Y_test.append('No')

In [5]:
filename = './DiplomaDataPerson_X.joblib.pkl'
joblib.dump(X, filename, compress=3)

['./DiplomaDataPerson_X.joblib.pkl']

In [6]:
filename = './DiplomaDataPerson_X_test.joblib.pkl'
joblib.dump(X_test, filename, compress=3)

['./DiplomaDataPerson_X_test.joblib.pkl']

In [14]:
set_futures =[]
for i in range(27):
    set_futures.append(set())
for x in X+X_test:
    for i, x_i in enumerate(x):
        set_futures[i].add(x_i)

In [15]:
feature_dict = []
for i in range(27):
    feature_dict.append(dict())
    
for i in range(27):
    count = 0
    for x in set_futures[i]:
        feature_dict[i][x] = count
        count+=1

In [16]:
X_int=[]
for x in X:
    X_int.append(x) 
for i, x in enumerate(X):
    for j, x_j in enumerate(x):
        X_int[i][j]=feature_dict[j][x_j]

In [17]:
X_int_test=[]
for x in X_test:
    X_int_test.append(x) 
for i, x in enumerate(X_test):
    for j, x_j in enumerate(x):
        X_int_test[i][j]=feature_dict[j][x_j]

In [18]:
filename = './DiplomaDataPerson_X_int.joblib.pkl'
joblib.dump(X_int, filename, compress=3)
filename = './DiplomaDataPerson_X_int_test.joblib.pkl'
joblib.dump(X_int_test, filename, compress=3)

['./DiplomaDataPerson_X_int_test.joblib.pkl']

In [19]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [20]:
X_big_int = X_int + X_int_test
print X_big_int[:2]

[[3, 5985, 15047, 352, 1, 1, 0, 2438, 1, 3, 33136, 3120, 2256, 1, 1, 1, 1343, 26, 4, 50165, 5680, 169, 1, 1, 1, 160, 20], [0, 61066, 9439, 1402, 1, 1, 0, 2438, 1, 2, 12331, 9889, 898, 1, 0, 1, 1423, 23, 10, 32273, 4578, 860, 1, 0, 1, 1479, 27]]


In [21]:
enc.fit_transform(X_big_int)

<894375x387382 sparse matrix of type '<type 'numpy.float64'>'
	with 24148125 stored elements in Compressed Sparse Row format>

In [22]:
X_sparse_matrix = enc.transform(X_int[:200000])

In [25]:
X_sparse_matrix_test = enc.transform(X_int_test[:200000])

In [26]:
X_sparse_matrix_test.shape

(200000, 387382)

In [29]:
import sklearn.ensemble  
RF = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=-1)
RF_plain = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=-1)

In [30]:
X_new = RF_plain.fit_transform(X_sparse_matrix, Y[:200000])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [31]:
from sklearn.externals import joblib
filename = './DiplomaDataPerson_RF_50_200ka_baddata.joblib.pkl'
joblib.dump(RF, filename, compress=3)

['./DiplomaDataPerson_RF_50_200ka_baddata.joblib.pkl']

DONE!!!

In [116]:
Y_test_proba =RF_plain.predict_proba(enc.transform(X_int[96946:]))

In [117]:
k = 0
for i, y in enumerate(Y_test_proba):
    if y[1] > 0.9:
        Y[96946+i] = 'Person'
        k+=1
print k

8509


In [94]:
from sklearn.externals import joblib
filename = './DiplomaDataPerson_Y_train.joblib.pkl'
joblib.dump(Y, filename, compress=3)

['./DiplomaDataPerson_Y_train.joblib.pkl']

In [128]:
Y_test_result =RF_plain.predict_proba(enc.transform(X_int_test))

In [134]:
k=0
for y in Y_test_result:
    if y[1] >0.1:
        k+=1
print k

286095


[['2', 'Neil', 'eil', 'il', 'False', 'True', 'False', 'Xx', 'NNP', '-1', '(', '(', '(', 'False', 'False', 'False', '(', '(', '1.0', 'Ferguson', 'son', 'on', 'False', 'True', 'False', 'Xx', 'NNP'], ['-1', 'Neil', 'eil', 'il', 'False', 'True', 'False', 'Xx', 'NNP', '-1', 'wa', 'was', 'as', 'False', 'False', 'False', 'x', 'VBD', '-1.0', 'career', 'eer', 'er', 'False', 'False', 'False', 'x', 'NN'], ['-1', 'Neil', 'eil', 'il', 'False', 'True', 'False', 'Xx', 'NNP', '-1', 'pass', 'sed', 'ed', 'False', 'False', 'False', 'x', 'VBD', '2.0', ',', ',', ',', 'False', 'False', 'False', ',', ','], ['-1', 'Neil', 'eil', 'il', 'False', 'True', 'False', 'Xx', 'NNP', '-1', 'attend', 'ded', 'ed', 'False', 'False', 'False', 'x', 'VBD', '2.0', ',', ',', ',', 'False', 'False', 'False', ',', ','], ['-1', 'Vinc', 'nce', 'ce', 'False', 'True', 'False', 'Xx', 'NNP', '-1', 'Cabl', 'ble', 'le', 'False', 'True', 'False', 'Xx', 'NNP', '-1.0', 'by', 'by', 'by', 'False', 'False', 'False', 'x', 'IN'], ['-1', 'Cabl', '

In [100]:
filename = './DiplomaDataPerson_X_test.joblib.pkl'
X_test = joblib.load(filename)

In [110]:
f = open('./personWithLabelNoTest', 'w')
f_n = 0
f_p = 0
t_n=0
t_p=0
for i, y in enumerate(Y_test_result):
    if y != Y_test[i]:
        if y=='No':
            f_n+=1
            #print X[10000+i]
        else:
            f_p+=1
    elif y =='No':
        t_n+=1 
        for x in X_test[i]:
            f.write(x)
            f.write(',')
        f.write('\n')
    else:
        t_p+=1
f.close()
recall = t_p*1./(t_p+f_n+1)
presision = t_p*1./(t_p+f_p+1)
print recall
print presision
print f_p, f_n, t_p, t_n
    

0.0
0.0
0 46850 0 239245


In [ ]:
F_measure = 2*presision*recall/(presision+recall)
print F_measure

In [ ]:
scores = cross_validation.cross_val_score(RF, X_sparse_matrix, Y[:300000], cv = 9,n_jobs=-1)

TO DO